In [1]:
import xarray as xr
import numpy as np
from pathlib import Path

In [16]:
data_path = Path("data/sat1/stage_data.nc")
stage_data = xr.load_dataset(data_path)
label_info = stage_data.labels.to_numpy()

In [13]:
label_info[0, 3, :]

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', ''], dtype=object)

In [17]:
dimensions = label_info.shape
for participant in range(dimensions[0]):
    for epoch in range(dimensions[1]):
        trial = label_info[participant, epoch, :]
        # If trial contains anything other than empty strings
        if not np.all(trial == ""):
            changes = np.where(trial[:-1] != trial[1:])[0] + 1
            trial_data = stage_data.isel(participant=participant, epochs=epoch)
            # Create list of EEG data splits
            splits = np.split(trial_data.data, changes, 1)
            label_splits = np.split(trial_data.labels, changes, 0)
            # print(splits)
            # print(label_splits)
            # Last split contains a value as first sample, instead of empty string/NaN
        else:
            print(f"Trial unusable: participant {participant}, epoch {epoch}")
            # for split in splits:
            # if np.any(np.isnan(split)):
            #     print(f"NAN FOUND participant {participant}, epoch {epoch}")
            #     print(split)
            # break
            # print(changes)
            # splits = np.split(trial, changes, 0)
            # print(len(splits))

Trial unusable: participant 0, epoch 0
Trial unusable: participant 0, epoch 1
Trial unusable: participant 0, epoch 3
Trial unusable: participant 0, epoch 5
Trial unusable: participant 0, epoch 8
Trial unusable: participant 0, epoch 9
Trial unusable: participant 0, epoch 10
Trial unusable: participant 0, epoch 13
Trial unusable: participant 0, epoch 14
Trial unusable: participant 0, epoch 16
Trial unusable: participant 0, epoch 18
Trial unusable: participant 0, epoch 20
Trial unusable: participant 0, epoch 21
Trial unusable: participant 0, epoch 25
Trial unusable: participant 0, epoch 26
Trial unusable: participant 0, epoch 27
Trial unusable: participant 0, epoch 29
Trial unusable: participant 0, epoch 30
Trial unusable: participant 0, epoch 32
Trial unusable: participant 0, epoch 34
Trial unusable: participant 0, epoch 39
Trial unusable: participant 0, epoch 40
Trial unusable: participant 0, epoch 41
Trial unusable: participant 0, epoch 42
Trial unusable: participant 0, epoch 44
Trial 

In [125]:
label_info[0, 4, :]

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'confirmation', 'confirmation',
       'confirmation', 'confirmation', 'confirmation', 'confirmation',
       'confirmation', 'confirmation', 'confirmation', 'confirmation',
       'confirmation', 'confirmation', 'confirmation', 'confirmation',
       'confirmation', 'confirmati

In [122]:
indices.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [137]:
# TODO: Has potential, find out which axes to split and how
# Need to split the DATA, not the labels, so indices should be enough but we should subset samples from data, not from labels
split_arr = np.split(label_info, indices[2] + 1, axis=2)

In [113]:
print(len(split_arr))

11399


In [144]:
split_arr[2][0, 4, :]

array(['decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision',
       'decision', 'decision', 'decision', 'decision', 'decision'],
      dtype=object)

In [80]:
# indices
print(indices[0].shape)
print(indices[1].shape)
print(indices[2].shape)
for i, j, k in zip(indices[0], indices[1], indices[2]):
    print(label_info[i, j, k])
# Split based on indices? For CNN:
# Select participants before (stage_data.sel(participant=...))
# Then

(11398,)
(11398,)
(11398,)

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmation
response

encoding
decision
confirmatio

In [30]:
specific_labels[indices[0] : indices[1]]

array(['encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding', 'encoding', 'encoding', 'encoding', 'encoding',
       'encoding'], dtype=object)